In [2]:
#Importing Librabries


import cv2 
import numpy as np
import os 
import matplotlib.pyplot as plt
import time 
import mediapipe as mp


ModuleNotFoundError: No module named 'cv2'

In [2]:
mp_holistic=  mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable =False 
    results = model.process(image)
    image.flags.writeable =True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
##rendering Landmarks 
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    

In [5]:
def draw_landmarks2(image,result):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius =1 )
                              
                             
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,44,10),thickness=2,circle_radius =1 )
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,44,10),thickness=2,circle_radius =1 )
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,117,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,66,10),thickness=2,circle_radius =1 )
                              )
    
    

In [60]:
##Tracking Face and Hands
i = 0
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        #Using Mediapipe

        image, results = mediapipe_detection(frame, holistic)   
        
        #Drawing Ladmarks
        draw_landmarks2(image,results)
        
        
        cv2.imshow("frame" , image)
       # cv2.imwrite("image.jpg", image)
        if cv2.waitKey(1)==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
##Extracting Landmarks 
"""pose =[]
for res in results.pose_landmarks.landmark:
    test = np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)"""

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

test = extract_keypoints(results)
test.shape

NameError: name 'results' is not defined

In [80]:
data_path = os.path.join("mp_data/")
actions= np.array(['Thankyou', "Hello", "Which","To_save"])
#actions= np.array(["To_save"])

no_sequences =30 #30 Videos for each actions
sequence_length = 30 ##30 Frames


In [78]:
#Making Folders 

for action in actions:
    for s in range(no_sequences):
        try:
            os.makedirs(os.path.join(data_path, action, str(s)))
        except:
            pass


In [79]:
##collecting images

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for s in range(no_sequences):
            for i in range(sequence_length):
                
            
        
                ret, frame = cap.read()
                #Using Mediapipe

                image, results = mediapipe_detection(frame, holistic)   

                #Drawing Ladmarks
                draw_landmarks2(image,results)

                if i==0:
                    cv2.putText(image, "Starting", (120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image, "Collecting Fraes for  {} Video Number {}".format(action,s ), (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, "Collecting Frames for  {} Video Number {}".format(action,s), (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(data_path, action,str(s),str(i))
                np.save(npy_path,keypoints)
                cv2.imshow("frame" , image)

                # cv2.imwrite("image.jpg", image)
                if cv2.waitKey(1)==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [81]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num,label in enumerate(actions)}

label_map

{'Thankyou': 0, 'Hello': 1, 'Which': 2, 'To_save': 3}

In [82]:
sequences, labels = [],[]
for action in actions:
    for seq in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(data_path,action , str(seq),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
np.array(sequences).shape

(120, 30, 1662)

In [83]:
y = to_categorical(labels).astype(int)
X = np.array(sequences)
X.shape

(120, 30, 1662)

In [84]:
##SPlitting Data

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05)



In [85]:
y_train.shape

(114, 4)

In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [87]:
log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir)



In [88]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu',input_shape= (30,1662)))
model.add(LSTM(128,return_sequences= True, activation = 'relu'))
model.add(LSTM(64,return_sequences= False, activation = 'relu'))
model.add(Dense(64,activation= 'relu'))
model.add(Dense(32,activation= 'relu'))
model.add(Dense(actions.shape[0],activation= 'softmax'))

model.compile(optimizer= "adam", loss = "categorical_crossentropy",metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs = 120,callbacks= [tb_callback])

Train on 114 samples
Epoch 1/120
114/114 [==============================] - 3s 30ms/sample - loss: 1.8895 - categorical_accuracy: 0.2456
Epoch 2/120
114/114 [==============================] - 1s 6ms/sample - loss: 5.1910 - categorical_accuracy: 0.1842
Epoch 3/120
114/114 [==============================] - 1s 6ms/sample - loss: 2.2522 - categorical_accuracy: 0.2193
Epoch 4/120
114/114 [==============================] - 1s 6ms/sample - loss: 2.9403 - categorical_accuracy: 0.3684
Epoch 5/120
114/114 [==============================] - 1s 6ms/sample - loss: 2.0115 - categorical_accuracy: 0.1754
Epoch 6/120
114/114 [==============================] - 1s 6ms/sample - loss: 2.0290 - categorical_accuracy: 0.2982
Epoch 7/120
114/114 [==============================] - 1s 6ms/sample - loss: 1.2250 - categorical_accuracy: 0.3596
Epoch 8/120
114/114 [==============================] - 1s 6ms/sample - loss: 1.3521 - categorical_accuracy: 0.4386
Epoch 9/120
114/114 [==============================] - 1s 

In [89]:
res = model.predict(X_test)
print(res)


[[2.8531968e-03 6.4982436e-05 5.7496753e-02 9.3958515e-01]
 [1.8043153e-03 4.6110497e-05 3.7223071e-02 9.6092641e-01]
 [5.6907747e-02 7.7199645e-04 7.6205307e-01 1.8026720e-01]
 [2.0769945e-01 9.4497873e-04 7.8747678e-01 3.8787862e-03]
 [8.5451204e-05 2.8950114e-06 1.9769613e-03 9.9793470e-01]
 [2.3082343e-03 9.9765718e-01 3.4458910e-05 1.5960715e-07]]


In [94]:
print(actions[np.argmax(res[0])])

print(actions[np.argmax(y_test[0])])

To_save
To_save


In [103]:
model.save("model3.h5")

In [96]:
##MODEL EVALUATION

from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_train)

ytrue= np.argmax(y_train,axis =1 ).tolist()
yhat = np.argmax(yhat,axis = 1 ).tolist()

multilabel_confusion_matrix(ytrue, yhat)


array([[[83,  1],
        [ 5, 25]],

       [[85,  0],
        [ 1, 28]],

       [[81,  5],
        [ 1, 27]],

       [[86,  1],
        [ 0, 27]]], dtype=int64)

In [97]:
accuracy_score(ytrue , yhat)

0.9385964912280702

In [67]:
import tensorflow as tf
model = tf.keras.models.load_model("model3.h5")

In [104]:
##Final testing
import tensorflow as tf
sequences = []#concatenating 30 frames
sentences = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        #Using Mediapipe

        image, results = mediapipe_detection(frame, holistic)   
        
        #Drawing Ladmarks
        draw_landmarks2(image,results)
        
        ##making predictions
        
        keypoints = extract_keypoints(results)
        #sequences.insert(0,keypoints)
        sequences.append(keypoints)
        sequences = sequences[-30:]
        
        if len(sequences)==30:
            res = model.predict(np.expand_dims(sequences,axis = 0))[0]
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    print(actions[np.argmax(res)])
                    if len(sentences)>0:
                        if actions[np.argmax(res)]!=sentences[-1]:
                            sentences.append(actions[np.argmax(res)])
                            
                    else:
                        sentences.append(actions[np.argmax(res)])
            if len(sentences)>5:
                sentences = sentences[-5:]

        cv2.rectangle(image, (0,0),(640,40), (245,117,16),-1 )
        cv2.putText(image,"".join(sentences),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        cv2.imshow("frame" , image)
       # cv2.imwrite("image.jpg", image)
        if cv2.waitKey(1)==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Which
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Thankyou
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
H

In [ ]:
model1 = tf.model.